<a href="https://colab.research.google.com/github/Taehwan2/hanghaeAI/blob/main/RAG%EC%BD%94%EB%93%9C%EB%A6%AC%EB%B7%B0%20%EC%8B%AC%ED%99%94%EA%B0%80%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [35]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [36]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

In [42]:
import requests
from langchain.schema import Document
# Raw 파일 주소로 변경 (blob -> raw.githubusercontent.com)
url = "https://raw.githubusercontent.com/Taehwan2/ConcertReservation/main/concert/concert/src/main/java/com/example/concert/domain/concertdetail/service/ConcertDetailService.java"

code_text = requests.get(url).text
dosc = [Document(page_content=code_text)]
print(dosc)



[Document(metadata={}, page_content='package com.example.concert.domain.concertdetail.service;\n\nimport com.example.concert.Presentation.concert.model.concert.ConcertDetailReq;\nimport com.example.concert.Presentation.concert.model.concert.ConcertReq;\nimport com.example.concert.domain.concert.entity.Concert;\nimport com.example.concert.domain.concertdetail.entity.ConcertDetail;\nimport lombok.RequiredArgsConstructor;\nimport org.springframework.cache.annotation.CacheEvict;\nimport org.springframework.cache.annotation.CachePut;\nimport org.springframework.cache.annotation.Cacheable;\nimport org.springframework.stereotype.Service;\n\nimport java.util.List;\n\n@Service\n@RequiredArgsConstructor\npublic class ConcertDetailService {\n    private final ConcertDetailRepository concertDetailRepository;\n\n   //콘서트 옵션을 저장하는 코드\n    public ConcertDetail saveConcertDetail(ConcertDetail concertDetail){\n        return concertDetailRepository.saveConcertDetail(concertDetail);\n    }\n\n    //cach

In [43]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(dosc)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY)
)

In [44]:
retriever = vectorstore.as_retriever()

user_msg = "You are the best backend developer at Google give me code reivew"
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[HumanMessage(content='You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\nQuestion: You are the best backend developer at Google give me code reivew \nContext: package com.example.concert.domain.concertdetail.service;\n\nimport com.example.concert.Presentation.concert.model.concert.ConcertDetailReq;\nimport com.example.concert.Presentation.concert.model.concert.ConcertReq;\nimport com.example.concert.domain.concert.entity.Concert;\nimport com.example.concert.domain.concertdetail.entity.ConcertDetail;\nimport lombok.RequiredArgsConstructor;\nimport org.springframework.cache.annotation.CacheEvict;\nimport org.springframework.cache.annotation.CachePut;\nimport org.springframework.cache.annotation.Cacheable;\nimport org.springframework.stereotype.Service;\n\nimport java.util.List;\n\n@Service\n

In [45]:
response = llm.invoke(user_prompt)
print(response.content)

The code contains a duplicate method `saveConcertDetail` which should be removed to avoid redundancy. Additionally, make sure the method names clearly reflect their functionality; using "save" for both adding and updating concert details can be misleading. Finally, ensure that all methods properly handle potential exceptions, especially for database operations.
